In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

import plotly.io as pio
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [2]:
%matplotlib inline

In [3]:
stock_files = [
    {'file_path': 'data/AAPL.csv', 'stock_name': 'AAPL'},
    {'file_path': 'data/AMC.csv', 'stock_name': 'AMC'},
    {'file_path': 'data/AMZN.csv', 'stock_name': 'AMZN'},
    {'file_path': 'data/BB.csv', 'stock_name': 'BB'},
    {'file_path': 'data/GME.csv', 'stock_name': 'GME'},
    {'file_path': 'data/GOOG.csv', 'stock_name': 'GOOG'},
    {'file_path': 'data/MSFT.csv', 'stock_name': 'MSFT'},
    {'file_path': 'data/PLTR.csv', 'stock_name': 'PLTR'},
    {'file_path': 'data/RIVN.csv', 'stock_name': 'RIVN'},
    {'file_path': 'data/SOFI.csv', 'stock_name': 'SOFI'},
    {'file_path': 'data/TSLAcsv', 'stock_name': 'TSLA'}
]

In [4]:
# Load the selected stock data
selected_stock_data = None

# UI Components
stock_dropdown = widgets.Dropdown(
    options=[(stock['stock_name'], i) for i, stock in enumerate(stock_files)],
    description='Select Stock:',
    disabled=False,
)

start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    disabled=False
)

# Set default values for price and volume range sliders
default_min_close = 0 
default_max_close = 100  

price_range_slider = widgets.FloatRangeSlider(
    value=[default_min_close, default_max_close],
    min=default_min_close,
    max=default_max_close,
    step=0.1,
    description='Price Range:',
    continuous_update=False
)

chart_type_dropdown = widgets.Dropdown(
    options=['Line Chart', 'Candlestick Chart'],
    value='Line Chart',
    description='Chart Type:',
    disabled=False,
)

def load_selected_stock_data(change):
    global selected_stock_data
    stock_index = change.new
    if stock_index is not None:
        file_path = stock_files[stock_index]['file_path']
        selected_stock_data = pd.read_csv(file_path)
        selected_stock_data['Date'] = pd.to_datetime(selected_stock_data['Date'])
        
        # Set the start and end date pickers to the dataset's start and end dates
        if not selected_stock_data.empty:
            min_date = selected_stock_data['Date'].min()
            max_date = selected_stock_data['Date'].max()
            start_date_picker.value = min_date.date()
            end_date_picker.value = max_date.date()

        # Calculate Buy and Sell signals
        selected_stock_data['Buy'] = selected_stock_data['Close'] > selected_stock_data['Open']
        selected_stock_data['Sell'] = selected_stock_data['Close'] < selected_stock_data['Open']

# Create labels for the widgets
stock_label = widgets.Label(value='Select Stock:')
date_range_label = widgets.Label(value='Date Range:')
price_range_label = widgets.Label(value='Price Range:')
chart_type_label = widgets.Label(value='Chart Type:')
apply_filters_label = widgets.Label(value='Apply Filters:')

# Observers and UI Components
stock_dropdown.observe(load_selected_stock_data, names='value')

date_range_box = widgets.HBox([start_date_picker, end_date_picker])
price_range_box = widgets.HBox([price_range_slider])

chart_options_box = widgets.HBox([chart_type_dropdown])

filter_button = widgets.Button(description='Apply Filters')
output = widgets.Output()

# Function to filter and display data
def filter_data(b):
    with output:
        # Close the previous figure
        # plt.close()
        plt.clf()
        clear_output(wait=True)
        
        if selected_stock_data is None or selected_stock_data.empty:
            print("Please select a stock and load the data.")
            return

        filtered_data = selected_stock_data[
            (selected_stock_data['Date'] >= pd.to_datetime(start_date_picker.value)) & 
            (selected_stock_data['Date'] <= pd.to_datetime(end_date_picker.value)) &
            (selected_stock_data['Close'] >= price_range_slider.value[0]) & 
            (selected_stock_data['Close'] <= price_range_slider.value[1])
        ]

        if filtered_data.empty:
            print("No data to display for the selected filters.")
            return
        
        # Display Buy and Sell signals
        buy_signals = filtered_data[filtered_data['Buy']]
        sell_signals = filtered_data[filtered_data['Sell']]

        if chart_type_dropdown.value == 'Line Chart':
            plt.figure(figsize=(10, 4))
            plt.plot(filtered_data['Date'], filtered_data['Close'], label='Close Price')

            # Plot Buy/Sell signals
            plt.scatter(buy_signals['Date'], buy_signals['Close'], marker='^', color='green', label='Buy Signal', alpha=1)
            plt.scatter(sell_signals['Date'], sell_signals['Close'], marker='v', color='red', label='Sell Signal', alpha=1)

            plt.xlabel('Date')
            plt.ylabel('Price')
            plt.title(f"{stock_files[stock_dropdown.value]['stock_name']} Stock Price")
            plt.legend()
            plt.show()

        elif chart_type_dropdown.value == 'Candlestick Chart':
            # Clear the Plotly figure if it exists
            if 'fig' in locals():
                fig = None

            fig = go.Figure(data=[go.Candlestick(
                x=filtered_data['Date'],
                open=filtered_data['Open'],
                high=filtered_data['High'],
                low=filtered_data['Low'],
                close=filtered_data['Close']
            )])

            fig.update_layout(
                title=f"{stock_files[stock_dropdown.value]['stock_name']} Candlestick Chart",
                title_font=dict(color='white'),
                title_x=0.5,  
                title_y=0.95, 
                xaxis_title='Date',
                yaxis_title='Price',
                xaxis_rangeslider_visible=False,
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(245, 245, 245, 1)',
                margin=dict(l=40, r=40, t=40, b=40),
                hovermode='closest',
                xaxis=dict(
                    showline=True,
                    showgrid=False,
                    showticklabels=True,
                    tickfont=dict(color='white'),
                    title_font=dict(color='white')
                ),
                yaxis=dict(
                    showline=True,
                    showgrid=True,
                    showticklabels=True,
                    gridcolor='gray',
                    tickfont=dict(color='white'),
                    title_font=dict(color='white')
                ),
                showlegend=True,
                autosize=True,
                legend_title_font=dict(color='white'),  # Set legend title color to white
                legend_font=dict(color='white')
            )
            
            # Add Buy and Sell markers or annotations
            fig.add_trace(go.Scatter(x=buy_signals['Date'], y=buy_signals['Close'], mode='markers', name='Buy', marker=dict(color='green', size=10, symbol='triangle-up')))
            fig.add_trace(go.Scatter(x=sell_signals['Date'], y=sell_signals['Close'], mode='markers', name='Sell', marker=dict(color='red', size=10, symbol='triangle-down')))

            fig.show()

# Link the filter button to the filter_data function
filter_button.on_click(filter_data)

# Create a VBox to organize the widgets vertically
ui = widgets.VBox([
    widgets.HBox([stock_label, stock_dropdown]),
    widgets.HBox([date_range_label, date_range_box]),
    widgets.HBox([chart_type_label, chart_options_box]),
    widgets.HBox([apply_filters_label, filter_button]),
    output,
])

display(ui)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [5]:
stock_label = widgets.Label(value='Select Stock:')
date_range_label = widgets.Label(value=f'<i class="fa-solid fa-calendar-days"></i>')
price_range_label = widgets.Label(value=f'<i class="fa-solid fa-dollar-sign"></i>')
volume_range_label = widgets.Label(value=f'fa-solid fa-arrow-trend-up"></i>')
chart_type_label = widgets.Label(value=f'<i class="fa-solid fa-chart-simple"></i>')
apply_filters_label = widgets.Label(value=f'<i class="fas fa-filter"></i>')

display(stock_label)
display(date_range_label)
display(price_range_label)
display(volume_range_label)
display(chart_type_label)
display(apply_filters_label)


Label(value='Select Stock:')

Label(value='<i class="fa-solid fa-calendar-days"></i>')

Label(value='<i class="fa-solid fa-dollar-sign"></i>')

Label(value='fa-solid fa-arrow-trend-up"></i>')

Label(value='<i class="fa-solid fa-chart-simple"></i>')

Label(value='<i class="fas fa-filter"></i>')

In [6]:
from IPython.display import display, HTML

stock_label = HTML(value='<span style="font-family: FontAwesome;"><i class="fa-solid fa-calendar-days"></i></span> Select Stock:')
date_range_label = HTML(value='<span style="font-family: FontAwesome;"><i class="fa-solid fa-calendar-days"></i></span> Date Range:')
price_range_label = HTML(value='<span style="font-family: FontAwesome;"><i class="fa-solid fa-dollar-sign"></i></span> Price Range:')
volume_range_label = HTML(value='<span style="font-family: FontAwesome;"><i class="fa-solid fa-arrow-trend-up"></i></span> Volume Range:')
chart_type_label = HTML(value='<span style="font-family: FontAwesome;"><i class="fa-solid fa-chart-simple"></i></span> Chart Type:')
apply_filters_label = HTML(value='<span style="font-family: FontAwesome;"><i class="fas fa-filter"></i></span> Apply Filters:')

display(stock_label)
display(date_range_label)
display(price_range_label)
display(volume_range_label)
display(chart_type_label)
display(apply_filters_label)


TypeError: HTML.__init__() got an unexpected keyword argument 'value'